In [1]:
import math
import os
import re
import altair as alt
import numpy as np
import pandas as pd
import scipy.stats
import yaml
import neutcurve
import scipy.stats
print(f"Using `neutcurve` version {neutcurve.__version__}")
import sys

# allow more rows for Altair
_ = alt.data_transformers.disable_max_rows()

#import altair themes from /data/custom_analyses_data/theme.py and enable
import theme
alt.themes.register('main_theme', theme.main_theme)
alt.themes.enable('main_theme')
from pathlib import Path

repo_root = Path.cwd().parent
os.chdir(repo_root)
print(os.getcwd())
#print(os.listdir("raw_plate_reader"))

Using `neutcurve` version 2.1.0
/fh/fast/bloom_j/computational_notebooks/tmcmahon/2024/02_RSV/RSV_evolution_neut


In [2]:
df = pd.read_csv('01_data/other/2025.01.14_PsvTiter_Calculations.csv')
df['virus'] = df['virus'].str.strip()
print(df['virus'].unique())
df

['RSV Long' 'RSV A2' 'RSV B1' 'RSV A 1982' 'RSV A 2020' 'RSV B 1982'
 'RSV B 2024' 'No RSV' 'RSV B 2019' 'RSV B 1992' 'VSV-G']


,virus,target cell,replicate,dilution,RLU,uL,RLU/uL,date
0,RSV Long,293T-TIM1,1,128.000,2908000.0,0.445312,6.530246e+06,25.01.14
1,RSV Long,293T-TIM1,1,512.000,840700.0,0.111328,7.551551e+06,25.01.14
2,RSV Long,293T-TIM1,1,2048.000,238900.0,0.027832,8.583635e+06,25.01.14
3,RSV Long,293T-TIM1,1,128.000,3594000.0,0.445312,8.070737e+06,25.01.14
4,RSV Long,293T-TIM1,1,512.000,961300.0,0.111328,8.634835e+06,25.01.14
...,...,...,...,...,...,...,...,...
259,VSV-G,293T-TIM1,2,4000.000,41020000.0,0.025000,1.640800e+09,25.02.10
260,VSV-G,293T-TIM1,2,16000.000,8847000.0,0.006250,1.415520e+09,25.02.10
261,VSV-G,293T-TIM1,2,64000.000,2131000.0,0.001563,1.363840e+09,25.02.10
262,VSV-G,293T-TIM1,2,256000.000,564100.0,0.000391,1.444096e+09,25.02.10


In [3]:
df.head

<bound method NDFrame.head of         virus target cell  replicate      dilution         RLU        uL  \
0    RSV Long   293T-TIM1           1      128.000   2908000.0  0.445312   
1    RSV Long   293T-TIM1           1      512.000    840700.0  0.111328   
2    RSV Long   293T-TIM1           1     2048.000    238900.0  0.027832   
3    RSV Long   293T-TIM1           1      128.000   3594000.0  0.445312   
4    RSV Long   293T-TIM1           1      512.000    961300.0  0.111328   
..        ...         ...         ...          ...         ...       ...   
259     VSV-G   293T-TIM1           2     4000.000  41020000.0  0.025000   
260     VSV-G   293T-TIM1           2    16000.000   8847000.0  0.006250   
261     VSV-G   293T-TIM1           2    64000.000   2131000.0  0.001563   
262     VSV-G   293T-TIM1           2   256000.000    564100.0  0.000391   
263     VSV-G   293T-TIM1           2  1023999.476    227900.0  0.000098   

           RLU/uL      date  
0    6.530246e+06  25.01.14

In [4]:
# Define a function to configure the chart style
def configure_chart(chart):
    return chart.configure_axis(
        labelFontSize=14,
        titleFontSize=14,
        grid=False
    ).configure_view(
        strokeWidth=2
    )

# Define a selection for hover
hover = alt.selection_point(fields=["virus", "replicate"], nearest=True, on="mouseover", empty="none")

# Create the base chart
base = alt.Chart(df).encode(
    x=alt.X(
        'virus:N',
        title='Virus',
        axis=alt.Axis(labelAngle=45)
    ),
    y=alt.Y(
        'RLU/uL:Q',
        title='RLU/uL',
        scale=alt.Scale(type='log')  # Optional: Use log scale if needed
    ),
    color=alt.Color(
        'replicate:N',
        title='Replicate',
        legend=alt.Legend(title="Replicate")
    ),
    tooltip=[
        alt.Tooltip('virus:N', title='Virus'),
        alt.Tooltip('target cell:N', title='Target Cell'),
        alt.Tooltip('replicate:N', title='Replicate'),
        alt.Tooltip('dilution:Q', title='Dilution'),
        alt.Tooltip('RLU:Q', title='RLU'),
        alt.Tooltip('uL:Q', title='uL'),
        alt.Tooltip('RLU/uL:Q', title='RLU/uL')
    ]
)

# Points and hover logic
points = base.mark_point(size=80, filled=True).add_params(hover)

# Add a rule (highlight nearest point)
highlight = base.transform_filter(hover).mark_circle(size=200, color="red")

# Combine points and highlight, and facet by 'target cell'
faceted_chart = (
    (points + highlight)
    .properties(width=300, height=300)
    .facet(
        facet=alt.Facet('target cell:N', title='Target Cell')
    )
)

# Configure and display the chart
final_chart = configure_chart(faceted_chart)
final_chart


alt.FacetChart(...)

## add error to new df 

In [5]:
# Compute mean and error bars
df_agg = df.groupby(['virus', 'target cell']).agg(
    mean_RLU_uL=('RLU/uL', 'mean'),
    std_RLU_uL=('RLU/uL', 'std')  # Standard deviation for error bars
).reset_index()

# Calculate upper and lower limits
df_agg['lower_RLU_uL'] = df_agg['mean_RLU_uL'] - df_agg['std_RLU_uL']
df_agg['upper_RLU_uL'] = df_agg['mean_RLU_uL'] + df_agg['std_RLU_uL']

# Display the final aggregated DataFrame
df_agg.head()

,virus,target cell,mean_RLU_uL,std_RLU_uL,lower_RLU_uL,upper_RLU_uL
0,No RSV,293T,307.885926,188.202923,119.683003,496.088849
1,No RSV,293T-TIM1,2634.666667,1492.515347,1142.151320,4127.182014
2,RSV A 1982,293T,85642.000000,19506.341309,66135.658691,105148.341309
3,RSV A 1982,293T-TIM1,550613.333333,81944.487344,468668.845989,632557.820677
4,RSV A 2020,293T,348498.666667,47383.088639,301115.578028,395881.755306


## final plot fig 1

In [6]:
virus_order = [
    "No RSV", "RSV Long", "RSV A2", "RSV A 1982", "RSV A 2020", "RSV B1",
    "RSV B 1982", "RSV B 1992", "RSV B 2019", "RSV B 2024", "VSV-G"
]

print("Expected Order:", virus_order)
print("Unique Values in DataFrame:", df_agg['virus'].unique())

# Ensure consistent virus naming by stripping spaces
df_agg['virus'] = df_agg['virus'].str.strip()

# Convert to categorical with the correct order
df_agg['virus'] = pd.Categorical(df_agg['virus'], categories=virus_order, ordered=True)

# Sort the DataFrame manually
df_agg = df_agg.sort_values('virus')

print(set(df_agg['virus']) - set(virus_order))

Expected Order: ['No RSV', 'RSV Long', 'RSV A2', 'RSV A 1982', 'RSV A 2020', 'RSV B1', 'RSV B 1982', 'RSV B 1992', 'RSV B 2019', 'RSV B 2024', 'VSV-G']
Unique Values in DataFrame: ['No RSV' 'RSV A 1982' 'RSV A 2020' 'RSV A2' 'RSV B 1982' 'RSV B 1992'
 'RSV B 2019' 'RSV B 2024' 'RSV B1' 'RSV Long' 'VSV-G']
set()


In [7]:
import pandas as pd
import altair as alt

# Define custom virus order
virus_order = [
    "No RSV", "RSV Long", "RSV A2", "RSV A 1982", "RSV A 2020", "RSV B1",
    "RSV B 1982", "RSV B 1992", "RSV B 2019", "RSV B 2024", "VSV-G"
]

# Ensure category consistency and remove spaces
df_agg['virus'] = df_agg['virus'].str.strip()

# Create an explicit numerical order for sorting
virus_order_dict = {virus: i for i, virus in enumerate(virus_order)}
df_agg['virus_order'] = df_agg['virus'].map(virus_order_dict)

# Sort DataFrame before passing it to Altair
df_agg = df_agg.sort_values('virus_order')

# Define color and shape mappings
color_mapping = {
    "293T": "#E69F00",       # Orange
    "293T-TIM1": "#999999"   # Gray
}

shape_mapping = {
    "293T": "square",
    "293T-TIM1": "circle"
}

# Define tick positions
log_ticks = [10**i for i in range(2, 8)]  # 10^2 to 10^7

# Manually define labels using Unicode superscripts
log_labels = ["10²", "10³", "10⁴", "10⁵", "10⁶", "10⁷"]

# Format label expression correctly for JavaScript
label_expr = "{ " + ", ".join(f"{i}: '{v}'" for i, v in enumerate(virus_order)) + " }[datum.value]"

# Base chart with explicit sorting
base = alt.Chart(df_agg).encode(
    x=alt.X(
        'virus_order:O',  # Use numerical order for sorting
        title='',
        axis=alt.Axis(
            labelAngle=270,
            labelFontSize=14,
            labelFontWeight='bold',
            titleFontSize=16,
            titleFontWeight='bold',
            values=list(range(len(virus_order))),  # Ensures Altair keeps labels
            labelExpr=label_expr  # Maps back to virus names
        )
    ),
     y=alt.Y(
        'mean_RLU_uL:Q',
        title='Pseudovirus Titer (RLU/uL)',
        scale=alt.Scale(type='log', domain=[100, df_agg['upper_RLU_uL'].max()]),
        axis=alt.Axis(
            labelFontSize=14,
            labelFontWeight='bold',
            titleFontSize=16,
            titleFontWeight='bold',
            values=log_ticks,  # Force log ticks
            labelExpr="{ " + ", ".join(f"{v}: '{label}'" for v, label in zip(log_ticks, log_labels)) + " }[datum.value]"
        ),
    ),
    color=alt.Color(
        'target cell:N',
        scale=alt.Scale(domain=list(color_mapping.keys()), range=list(color_mapping.values())),
        legend=alt.Legend(title='Target Cell')  # KEEP LEGEND
    ),
    shape=alt.Shape(
        'target cell:N',
        scale=alt.Scale(domain=list(shape_mapping.keys()), range=list(shape_mapping.values())),
        legend=alt.Legend(title='Target Cell')  # KEEP LEGEND
    ),
    tooltip=[
        alt.Tooltip('virus:N', title='Virus'),
        alt.Tooltip('target cell:N', title='Target Cell'),
        alt.Tooltip('mean_RLU_uL:Q', title='Pseudovirus Titer (RLU/uL)'),
        alt.Tooltip('std_RLU_uL:Q', title='Standard Deviation')
    ]
)

# Error bars with black color (added first to appear behind)
error_bars = alt.Chart(df_agg).mark_rule(size=2, color='black').encode(
    x='virus_order:O',
    y='lower_RLU_uL:Q',
    y2='upper_RLU_uL:Q'
)

# Error bar caps with black color (added before points to stay behind)
error_caps = (
    alt.Chart(df_agg).mark_tick(size=12, thickness=2, orient='horizontal', color='black').encode(
        x='virus_order:O',
        y='lower_RLU_uL:Q'
    ) +
    alt.Chart(df_agg).mark_tick(size=12, thickness=2, orient='horizontal', color='black').encode(
        x='virus_order:O',
        y='upper_RLU_uL:Q'
    )
)

# Points for the mean with fixed shape encoding and black outline (added last to stay on top)
points = base.mark_point(size=80, filled=True, opacity=1, stroke='black', strokeWidth=1.5).add_params(hover)

# Highlight nearest point (still on top)
highlight = base.transform_filter(hover).mark_circle(size=200, color="red")

# Combine layers ensuring error bars & caps are behind points
combined_chart = (error_bars + error_caps + points + highlight).properties(
    width=300,
    height=200
).configure_axis(
    labelFontSize=8,  # Apply globally to be safe
    labelFontWeight='bold',
    titleFontSize=12,
    titleFontWeight='bold',
    grid=False
).configure_view(
    strokeWidth=2
)

combined_chart.save("03_output/plots/Titer_RLU-uL_wVSVG.html")

# Display the chart
combined_chart

alt.LayerChart(...)

In [8]:
df_agg = df_agg[df_agg['virus'] != 'VSV-G']


# Define custom virus order
virus_order = [
    "No RSV", "RSV Long", "RSV A2", "RSV A 1982", "RSV A 2020", "RSV B1",
    "RSV B 1982", "RSV B 1992", "RSV B 2019", "RSV B 2024"
]

# Ensure category consistency and remove spaces
df_agg['virus'] = df_agg['virus'].str.strip()

# Create an explicit numerical order for sorting
virus_order_dict = {virus: i for i, virus in enumerate(virus_order)}
df_agg['virus_order'] = df_agg['virus'].map(virus_order_dict)

# Sort DataFrame before passing it to Altair
df_agg = df_agg.sort_values('virus_order')

# Define color and shape mappings
color_mapping = {
    "293T": "#E69F00",       # Orange
    "293T-TIM1": "#999999"   # Gray
}

shape_mapping = {
    "293T": "square",
    "293T-TIM1": "circle"
}

# Define tick positions
log_ticks = [10**i for i in range(2, 8)]  # 10^2 to 10^7

# Manually define labels using Unicode superscripts
log_labels = ["10²", "10³", "10⁴", "10⁵", "10⁶", "10⁷"]

# Format label expression correctly for JavaScript
label_expr = "{ " + ", ".join(f"{i}: '{v}'" for i, v in enumerate(virus_order)) + " }[datum.value]"

# Base chart with explicit sorting
base = alt.Chart(df_agg).encode(
    x=alt.X(
        'virus_order:O',  # Use numerical order for sorting
        title='',
        axis=alt.Axis(
            labelAngle=270,
            labelFontSize=14,
            labelFontWeight='bold',
            titleFontSize=16,
            titleFontWeight='bold',
            values=list(range(len(virus_order))),  # Ensures Altair keeps labels
            labelExpr=label_expr  # Maps back to virus names
        )
    ),
     y=alt.Y(
        'mean_RLU_uL:Q',
        title='Pseudovirus Titer (RLU/uL)',
        scale=alt.Scale(type='log', domain=[100, df_agg['upper_RLU_uL'].max()]),
        axis=alt.Axis(
            labelFontSize=14,
            labelFontWeight='bold',
            titleFontSize=16,
            titleFontWeight='bold',
            values=log_ticks,  # Force log ticks
            labelExpr="{ " + ", ".join(f"{v}: '{label}'" for v, label in zip(log_ticks, log_labels)) + " }[datum.value]"
        ),
    ),
    color=alt.Color(
        'target cell:N',
        scale=alt.Scale(domain=list(color_mapping.keys()), range=list(color_mapping.values())),
        legend=alt.Legend(title='Target Cell')  # KEEP LEGEND
    ),
    shape=alt.Shape(
        'target cell:N',
        scale=alt.Scale(domain=list(shape_mapping.keys()), range=list(shape_mapping.values())),
        legend=alt.Legend(title='Target Cell')  # KEEP LEGEND
    ),
    tooltip=[
        alt.Tooltip('virus:N', title='Virus'),
        alt.Tooltip('target cell:N', title='Target Cell'),
        alt.Tooltip('mean_RLU_uL:Q', title='Pseudovirus Titer (RLU/uL)'),
        alt.Tooltip('std_RLU_uL:Q', title='Standard Deviation')
    ]
)

# Error bars with black color (added first to appear behind)
error_bars = alt.Chart(df_agg).mark_rule(size=2, color='black').encode(
    x='virus_order:O',
    y='lower_RLU_uL:Q',
    y2='upper_RLU_uL:Q'
)

# Error bar caps with black color (added before points to stay behind)
error_caps = (
    alt.Chart(df_agg).mark_tick(size=12, thickness=2, orient='horizontal', color='black').encode(
        x='virus_order:O',
        y='lower_RLU_uL:Q'
    ) +
    alt.Chart(df_agg).mark_tick(size=12, thickness=2, orient='horizontal', color='black').encode(
        x='virus_order:O',
        y='upper_RLU_uL:Q'
    )
)

# Points for the mean with fixed shape encoding and black outline (added last to stay on top)
points = base.mark_point(size=80, filled=True, opacity=1, stroke='black', strokeWidth=1.5).add_params(hover)

# Highlight nearest point (still on top)
highlight = base.transform_filter(hover).mark_circle(size=200, color="red")

# Combine layers ensuring error bars & caps are behind points
combined_chart = (error_bars + error_caps + points + highlight).properties(
    width=300,
    height=200
).configure_axis(
    labelFontSize=8,  # Apply globally to be safe
    labelFontWeight='bold',
    titleFontSize=12,
    titleFontWeight='bold',
    grid=False
).configure_view(
    strokeWidth=2
)

combined_chart.save("03_output/plots/Titer_RLU-uL.html")

# Display the chart
combined_chart

/tmp/ipykernel_29490/2203146847.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_agg['virus'] = df_agg['virus'].str.strip()
/tmp/ipykernel_29490/2203146847.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_agg['virus_order'] = df_agg['virus'].map(virus_order_dict)


alt.LayerChart(...)